In [719]:
import pandas as pd

# Load data

In [720]:
awards_players_df = pd.read_csv('../data/awards_players.csv'); awards_players_df.Name = "awards_players"
coaches_df = pd.read_csv('../data/coaches.csv'); coaches_df.Name = "coaches"
players_teams_df = pd.read_csv('../data/players_teams.csv'); players_teams_df.Name = "players_teams"
players_df = pd.read_csv('../data/players.csv'); players_df.Name = "players"
series_post_df = pd.read_csv('../data/series_post.csv'); series_post_df.Name = "series_post"
teams_post_df = pd.read_csv('../data/teams_post.csv'); teams_post_df.Name = "teams_post"
teams_df = pd.read_csv('../data/teams.csv'); teams_df.Name = "teams"

dfs = [awards_players_df, coaches_df, players_teams_df, players_df, series_post_df, teams_post_df, teams_df]

# Clean data

### Drop noisy data

In [721]:
players_df.drop(players_df[players_df['height'] <= 0].index, inplace = True)

### Remove duplicates


In [722]:
for df in dfs:
    df.drop_duplicates(inplace=True)
    display(f"Dropping dupplicates from dataframe {df.Name}...")

'Dropping dupplicates from dataframe awards_players...'

'Dropping dupplicates from dataframe coaches...'

'Dropping dupplicates from dataframe players_teams...'

'Dropping dupplicates from dataframe players...'

'Dropping dupplicates from dataframe series_post...'

'Dropping dupplicates from dataframe teams_post...'

'Dropping dupplicates from dataframe teams...'

### Drop unnecessary columns

In [723]:
players_df.drop(['birthDate', 'deathDate', 'college', 'collegeOther'], axis=1, inplace=True)
teams_df.drop(["confID", "franchID", "firstRound", "semis", "finals", "attend", "name", "arena"], axis=1, inplace=True);
teams_post_df.drop([], axis=1, inplace=True)
series_post_df.drop([], axis=1, inplace=True)
players_teams_df.drop([], axis=1, inplace=True)
awards_players_df.drop(['award'], axis=1, inplace=True)
coaches_df.drop([], axis=1, inplace=True)

### Drop single value columns

In [724]:
def drop_single_value_columns(df):
    for column in df.columns:
        if len(df[column].unique()) == 1:
            display(f"Dropping column {column} from datarame {df.Name}")

            df.drop(column, axis=1, inplace=True)

for df in dfs:
    drop_single_value_columns(df)

'Dropping column lgID from datarame awards_players'

'Dropping column lgID from datarame coaches'

'Dropping column lgID from datarame players_teams'

'Dropping column firstseason from datarame players'

'Dropping column lastseason from datarame players'

...

'Dropping column tmDRB from datarame teams'

'Dropping column tmTRB from datarame teams'

'Dropping column opptmORB from datarame teams'

'Dropping column opptmDRB from datarame teams'

'Dropping column opptmTRB from datarame teams'

### Drop rows with missing values

In [725]:
players_df.drop(players_df[players_df['height'] == 0].index, inplace = True)
players_df.drop(players_df[players_df['weight'] == 0].index, inplace = True)

### Rename columns and replace values

In [726]:
players_df.rename(columns={'bioID': 'playerID', 'stint': 'player_stint'}, inplace=True)
players_df.replace('F-C', 'C-F', inplace=True)
players_df.replace('F-G', 'G-F', inplace=True)

coaches_df.rename(columns={'won': 'coach_wins', 'lost': 'coach_losses', 'post_wins': 'coach_post_wins', 'post_losses': 'coach_post_losses', 'stint': 'coach_stint'}, inplace=True)

teams_df.rename(columns={'GP': 'team_GP', 'rank': 'current_year_rank'}, inplace=True)

players_teams_df.rename(columns={'GP': 'player_team_GP'}, inplace=True)

teams_post_df.rename(columns={'W': 'team_post_wins', 'L': 'team_post_losses'}, inplace=True)

series_post_df.rename(columns={'W': 'series_post_wins', 'L': 'series_post_lost'}, inplace=True)

### Add new columns

In [727]:
awards_players_df['award'] = True

for tmID, team_data in teams_df.groupby('tmID'):
    for year in team_data['year'].values:
        previous_year = year - 1

        previous_year_data = team_data[team_data['year'] == previous_year]

        # register previous year rank for each team
        previous_year_rank_series_data = previous_year_data['current_year_rank'].values
        previous_year_rank = previous_year_rank_series_data[0] if len(previous_year_rank_series_data) > 0 else -1
        teams_df.loc[((teams_df['tmID'] == tmID) & (team_data['year'] == year)), 'previous_year_rank'] = previous_year_rank

        # get players that played in the previous year for the team
        previous_year_player_ids = players_teams_df[(players_teams_df['tmID'] == tmID) & (players_teams_df['year'] == previous_year)]['playerID'].values
        previous_year_players = players_df[players_df['playerID'].isin(previous_year_player_ids)]

        # register average height and weight for each position for each team
        for position, players_per_position in previous_year_players.groupby('pos'):

            previous_year_avg_height = players_per_position['height'].mean()
            previous_year_avg_weight = players_per_position['weight'].mean()

            teams_df.loc[((teams_df['tmID'] == tmID) & (team_data['year'] == year)), f'previous_year_avg_height_pos_{position}'] = players_per_position['height'].mean()
            teams_df.loc[((teams_df['tmID'] == tmID) & (team_data['year'] == year)), f'previous_year_avg_weight_pos_{position}'] = players_per_position['weight'].mean()

        # count number of players awarded in the previous year
        # this gave the same accuracies but with a leakage somewhere
        previous_year_team_players_awards = awards_players_df[(awards_players_df['playerID'].isin(previous_year_player_ids)) & (awards_players_df['year'] == previous_year)]
        teams_df.loc[((teams_df['tmID'] == tmID) & (team_data['year'] == year)), 'previous_year_team_players_awarded'] = previous_year_team_players_awards['award'].count()

        # shift numerical stats one year forward
        for column in teams_df.loc[:,'o_fgm':'min']:
            teams_df.loc[((teams_df['tmID'] == tmID) & (team_data['year'] == year)), f'previous_year_{column}'] = previous_year_data[column].values[0] if len(previous_year_data[column].values) > 0 else -1

        # calculate AVG PPG for previous year
        previous_year_avg_ppg = previous_year_data['o_pts'].sum() / previous_year_data['team_GP'].sum()
        teams_df.loc[((teams_df['tmID'] == tmID) & (team_data['year'] == year)), 'previous_year_avg_ppg'] = previous_year_avg_ppg

        # register previous year coach stats
        previous_year_coach = coaches_df[(coaches_df['tmID'] == tmID) & (coaches_df['year'] == previous_year)]

# drop previous year data
teams_df.drop(teams_df.loc[:,'o_fgm':'min'], axis=1, inplace = True)

/tmp/ipykernel_1858622/1602640321.py:37: RuntimeWarning: invalid value encountered in scalar divide
  previous_year_avg_ppg = previous_year_data['o_pts'].sum() / previous_year_data['team_GP'].sum()
/tmp/ipykernel_1858622/1602640321.py:37: RuntimeWarning: invalid value encountered in scalar divide
  previous_year_avg_ppg = previous_year_data['o_pts'].sum() / previous_year_data['team_GP'].sum()


/tmp/ipykernel_1858622/1602640321.py:37: RuntimeWarning: invalid value encountered in scalar divide
  previous_year_avg_ppg = previous_year_data['o_pts'].sum() / previous_year_data['team_GP'].sum()
/tmp/ipykernel_1858622/1602640321.py:37: RuntimeWarning: invalid value encountered in scalar divide
  previous_year_avg_ppg = previous_year_data['o_pts'].sum() / previous_year_data['team_GP'].sum()
/tmp/ipykernel_1858622/1602640321.py:37: RuntimeWarning: invalid value encountered in scalar divide
  previous_year_avg_ppg = previous_year_data['o_pts'].sum() / previous_year_data['team_GP'].sum()
/tmp/ipykernel_1858622/1602640321.py:37: RuntimeWarning: invalid value encountered in scalar divide
  previous_year_avg_ppg = previous_year_data['o_pts'].sum() / previous_year_data['team_GP'].sum()
/tmp/ipykernel_1858622/1602640321.py:37: RuntimeWarning: invalid value encountered in scalar divide
  previous_year_avg_ppg = previous_year_data['o_pts'].sum() / previous_year_data['team_GP'].sum()
/tmp/ipyke

# Merge relevant data

In [728]:
main_df = teams_df

# Merge teams_df with coaches_df
# main_df = pd.merge(teams_df, coaches_df, on=['year', 'tmID'], how='left')

# Merge teams_df with teams_post_df
# main_df = pd.merge(main_df, teams_post_df, on=['year', 'tmID'], how='left')

# Merge with players_teams_df
# main_df = pd.merge(main_df, players_teams_df, on=['year', 'tmID'], how='left')

# Merge with awards_players_df
# main_df = pd.merge(main_df, awards_players_df, on=['playerID','year'], how='left')

# Merge with players_df
# main_df = pd.merge(main_df, players_df, on=['playerID'], how='left')

# Display the resulting merged data frame
main_df.head()

,year,tmID,current_year_rank,playoff,previous_year_rank,previous_year_team_players_awarded,previous_year_o_fgm,previous_year_o_fga,previous_year_o_ftm,previous_year_o_fta,...,previous_year_avg_height_pos_C,previous_year_avg_weight_pos_C,previous_year_avg_height_pos_C-F,previous_year_avg_weight_pos_C-F,previous_year_avg_height_pos_F,previous_year_avg_weight_pos_F,previous_year_avg_height_pos_G,previous_year_avg_weight_pos_G,previous_year_avg_height_pos_G-F,previous_year_avg_weight_pos_G-F
0,9,ATL,7,N,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,ATL,2,Y,7.0,0.0,895.0,2258.0,542.0,725.0,...,79.0,218.0,76.0,186.5,74.400000,175.200000,68.750000,147.250000,73.5,153.0
2,1,CHA,8,N,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,CHA,4,Y,8.0,0.0,812.0,1903.0,431.0,577.0,...,76.0,215.0,74.5,182.5,71.666667,154.666667,69.000000,158.200000,71.0,145.0
4,3,CHA,2,Y,4.0,0.0,746.0,1780.0,410.0,528.0,...,77.0,219.5,75.0,182.5,70.500000,156.500000,68.833333,142.666667,NaN,NaN


# Drop irrelevant columns
### After merging

In [729]:
# main_df = main_df.drop(["rank", "firstRound", "semis", "finals", "attend", "stint_x", "stint_y", "birthDate", "deathDate", "height", "weight", "award"], axis=1)
# main_df['award'].fillna(False, inplace=True) # mark non-award winners as False
# main_df['team_post_wins'].fillna(0, inplace=True)
# main_df['team_post_losses'].fillna(0, inplace=True)

for column in main_df.filter(regex=("previous_year_avg_.*")).columns.to_list():
    if column.startswith('previous_year_avg_height_pos_') or column.startswith('previous_year_avg_weight_pos_'):
        main_df[column].fillna(-1, inplace=True)

main_df['previous_year_avg_ppg'].fillna(-1, inplace=True)

display(main_df)

,year,tmID,current_year_rank,playoff,previous_year_rank,previous_year_team_players_awarded,previous_year_o_fgm,previous_year_o_fga,previous_year_o_ftm,previous_year_o_fta,...,previous_year_avg_height_pos_C,previous_year_avg_weight_pos_C,previous_year_avg_height_pos_C-F,previous_year_avg_weight_pos_C-F,previous_year_avg_height_pos_F,previous_year_avg_weight_pos_F,previous_year_avg_height_pos_G,previous_year_avg_weight_pos_G,previous_year_avg_height_pos_G-F,previous_year_avg_weight_pos_G-F
0,9,ATL,7,N,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.00,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000
1,10,ATL,2,Y,7.0,0.0,895.0,2258.0,542.0,725.0,...,79.0,218.0,76.00,186.5,74.400000,175.200000,68.750000,147.250000,73.5,153.000000
2,1,CHA,8,N,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.00,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000
3,2,CHA,4,Y,8.0,0.0,812.0,1903.0,431.0,577.0,...,76.0,215.0,74.50,182.5,71.666667,154.666667,69.000000,158.200000,71.0,145.000000
4,3,CHA,2,Y,4.0,0.0,746.0,1780.0,410.0,528.0,...,77.0,219.5,75.00,182.5,70.500000,156.500000,68.833333,142.666667,-1.0,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,6,WAS,5,N,4.0,0.0,873.0,2088.0,474.0,661.0,...,75.0,180.0,75.50,192.5,74.000000,168.333333,68.750000,137.500000,71.5,157.500000
138,7,WAS,4,Y,5.0,1.0,847.0,1968.0,388.0,546.0,...,75.0,180.0,75.50,192.5,72.500000,166.250000,67.200000,133.800000,71.0,160.000000
139,8,WAS,5,N,4.0,0.0,1016.0,2199.0,528.0,715.0,...,79.0,170.0,75.50,192.5,71.000000,171.500000,69.250000,154.250000,71.0,173.666667
140,9,WAS,6,N,5.0,0.0,877.0,2170.0,668.0,839.0,...,78.0,210.0,75.50,192.5,72.600000,172.400000,69.250000,154.250000,71.0,157.500000


# Export clean data to a .CSV file

In [730]:
main_df.to_csv("../data/clean/main_df.csv", index=False)
main_df.head(20)

,year,tmID,current_year_rank,playoff,previous_year_rank,previous_year_team_players_awarded,previous_year_o_fgm,previous_year_o_fga,previous_year_o_ftm,previous_year_o_fta,...,previous_year_avg_height_pos_C,previous_year_avg_weight_pos_C,previous_year_avg_height_pos_C-F,previous_year_avg_weight_pos_C-F,previous_year_avg_height_pos_F,previous_year_avg_weight_pos_F,previous_year_avg_height_pos_G,previous_year_avg_weight_pos_G,previous_year_avg_height_pos_G-F,previous_year_avg_weight_pos_G-F
0,9,ATL,7,N,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,10,ATL,2,Y,7.0,0.0,895.0,2258.0,542.0,725.0,...,79.000000,218.000000,76.000000,186.5,74.400000,175.200000,68.750000,147.250000,73.500000,153.000000
2,1,CHA,8,N,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
3,2,CHA,4,Y,8.0,0.0,812.0,1903.0,431.0,577.0,...,76.000000,215.000000,74.500000,182.5,71.666667,154.666667,69.000000,158.200000,71.000000,145.000000
4,3,CHA,2,Y,4.0,0.0,746.0,1780.0,410.0,528.0,...,77.000000,219.500000,75.000000,182.5,70.500000,156.500000,68.833333,142.666667,-1.000000,-1.000000
5,4,CHA,2,Y,2.0,0.0,770.0,1790.0,490.0,663.0,...,76.000000,204.666667,74.000000,187.5,70.500000,156.500000,68.333333,140.333333,-1.000000,-1.000000
6,5,CHA,5,N,2.0,0.0,787.0,1881.0,456.0,590.0,...,76.000000,199.000000,74.000000,202.5,71.666667,162.666667,68.800000,143.400000,-1.000000,-1.000000
7,6,CHA,6,N,5.0,0.0,745.0,1744.0,436.0,590.0,...,75.333333,183.000000,75.000000,220.0,72.200000,164.600000,69.200000,150.800000,-1.000000,-1.000000
8,7,CHA,6,N,6.0,0.0,772.0,1913.0,447.0,624.0,...,75.000000,202.000000,75.333333,185.0,73.500000,162.250000,68.500000,154.500000,72.000000,160.000000
9,7,CHI,7,N,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


### Information about the resulting dataset

In [731]:
main_df.info()
main_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 141
Data columns (total 57 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   year                                142 non-null    int64  
 1   tmID                                142 non-null    object 
 2   current_year_rank                   142 non-null    int64  
 3   playoff                             142 non-null    object 
 4   previous_year_rank                  142 non-null    float64
 5   previous_year_team_players_awarded  142 non-null    float64
 6   previous_year_o_fgm                 142 non-null    float64
 7   previous_year_o_fga                 142 non-null    float64
 8   previous_year_o_ftm                 142 non-null    float64
 9   previous_year_o_fta                 142 non-null    float64
 10  previous_year_o_3pm                 142 non-null    float64
 11  previous_year_o_3pa                 142 non-n

,year,current_year_rank,previous_year_rank,previous_year_team_players_awarded,previous_year_o_fgm,previous_year_o_fga,previous_year_o_ftm,previous_year_o_fta,previous_year_o_3pm,previous_year_o_3pa,...,previous_year_avg_height_pos_C,previous_year_avg_weight_pos_C,previous_year_avg_height_pos_C-F,previous_year_avg_weight_pos_C-F,previous_year_avg_height_pos_F,previous_year_avg_weight_pos_F,previous_year_avg_height_pos_G,previous_year_avg_weight_pos_G,previous_year_avg_height_pos_G-F,previous_year_avg_weight_pos_G-F
count,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,...,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000
mean,5.302817,4.084507,3.359155,0.521127,730.774648,1734.753521,413.591549,553.338028,132.908451,391.626761,...,56.751761,150.673357,56.500000,140.673709,62.439085,147.019249,58.641228,126.669470,48.311385,110.043192
std,2.917274,2.095226,2.666689,0.848180,307.196276,722.684929,179.821943,238.175121,67.044487,190.335337,...,32.788538,83.567337,33.012584,81.683360,26.535294,62.324182,24.250575,52.128826,33.721789,76.354116
min,1.000000,1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,3.000000,2.000000,1.000000,0.000000,762.250000,1822.750000,410.500000,552.250000,109.000000,332.750000,...,41.625000,165.500000,72.500000,170.625000,72.500000,164.000000,67.200000,141.750000,-1.000000,-1.000000
50%,5.000000,4.000000,3.000000,0.000000,828.000000,1947.000000,455.500000,617.500000,149.500000,431.000000,...,76.000000,187.000000,75.000000,184.250000,73.250000,171.000000,68.333333,146.666667,70.291667,155.000000
75%,8.000000,6.000000,6.000000,1.000000,895.750000,2110.750000,523.250000,688.000000,174.000000,518.500000,...,76.666667,198.916667,75.500000,190.000000,74.000000,176.125000,69.000000,150.787500,72.000000,165.000000
max,10.000000,8.000000,8.000000,4.000000,1079.000000,2485.000000,668.000000,882.000000,283.000000,802.000000,...,79.000000,240.000000,77.000000,220.000000,75.500000,207.500000,70.666667,159.500000,75.000000,185.500000
